# Create Study

Creates a new study record in the `md_study` table and a protocol draft record in `md_protocol_draft`.

## Parameters
- `catalog`: Catalog name
- `study_id`: Unique study identifier
- `study_title`: Study title
- `study_description`: Study description (optional)
- `protocol_id`: Protocol document ID
- `created_by`: User who created the study

## Output Tables
- `gold_md.md_study` - Study registry
- `silver_md.md_protocol_draft` - Protocol draft records


In [ ]:
# Widget parameters
dbutils.widgets.text("catalog", "dta_poc_test", "Catalog")
dbutils.widgets.text("study_id", "", "Study ID")
dbutils.widgets.text("study_title", "", "Study Title")
dbutils.widgets.text("study_nickname", "", "Study Nickname")
dbutils.widgets.text("study_description", "", "Study Description")
dbutils.widgets.text("study_phase", "", "Study Phase")
dbutils.widgets.text("protocol_id", "", "Protocol ID")
dbutils.widgets.text("created_by", "", "Created By")

# Get parameter values
catalog = dbutils.widgets.get("catalog")
study_id = dbutils.widgets.get("study_id")
study_title = dbutils.widgets.get("study_title")
study_nickname = dbutils.widgets.get("study_nickname")
study_description = dbutils.widgets.get("study_description")
study_phase = dbutils.widgets.get("study_phase")
protocol_id = dbutils.widgets.get("protocol_id")
created_by = dbutils.widgets.get("created_by")

# Get job context from setup task (for lineage tracking)
try:
    databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")
    databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
    databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
    print(f"Job context from setup task:")
    print(f"  Run ID: {databricks_run_id}")
    print(f"  Job ID: {databricks_job_id}")
    print(f"  Job Name: {databricks_job_name}")
except Exception as e:
    # Fallback for standalone testing
    from datetime import datetime
    databricks_run_id = "test_run_" + datetime.now().strftime("%Y%m%d_%H%M%S")
    databricks_job_id = "test_job"
    databricks_job_name = "nb_create_study"
    print(f"Running in standalone mode (no setup task): {e}")

print(f"\nCatalog: {catalog}")
print(f"Study ID: {study_id}")
print(f"Study Title: {study_title}")
print(f"Protocol ID: {protocol_id}")
print(f"Created By: {created_by}")


In [ ]:
# Validate required parameters
if not study_id:
    raise ValueError("study_id is required")
if not study_title:
    raise ValueError("study_title is required")

# Define schema and table
SCHEMA = "gold_md"
TABLE = "md_study"
full_table_name = f"{catalog}.{SCHEMA}.{TABLE}"
print(f"Target table: {full_table_name}")


In [ ]:
# Debug: Check if table exists BEFORE create
try:
    exists_before = spark.catalog.tableExists(full_table_name)
    print(f"Table exists BEFORE CREATE: {exists_before}")
    if exists_before:
        print("Existing schema:")
        spark.sql(f"DESCRIBE {full_table_name}").show(truncate=False)
except Exception as e:
    print(f"Table check error: {e}")

# Create table if not exists - includes audit columns for lineage
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {full_table_name} (
        study_id STRING NOT NULL COMMENT 'Unique study identifier',
        study_title STRING COMMENT 'Study title/name',
        study_description STRING COMMENT 'Study description',
        status STRING COMMENT 'Study status: IN_PROGRESS, PARSING_IN_PROGRESS, PARSING_COMPLETED, PARSING_FAILED, COMPLETED',
        protocol_id STRING COMMENT 'Associated protocol document ID',
        created_by_principal STRING COMMENT 'User who created the study',
        created_ts TIMESTAMP COMMENT 'Creation timestamp',
        last_updated_by_principal STRING COMMENT 'User who last updated the study',
        last_updated_ts TIMESTAMP COMMENT 'Last update timestamp',
        databricks_job_id STRING COMMENT 'Job ID for lineage',
        databricks_job_name STRING COMMENT 'Job name for lineage',
        databricks_run_id STRING COMMENT 'Run ID for lineage'
    ) USING DELTA
    COMMENT 'Study registry for clinical data management. '
""")
print(f"Table {full_table_name} ready")

# Debug: Print table schema after CREATE TABLE
print(f"\n=== Table Schema after CREATE TABLE ===")
spark.sql(f"DESCRIBE {full_table_name}").show(truncate=False)


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType
from datetime import datetime
from clinical_data_standards_framework.utils import save_with_audit

# Study schema - feature columns only (audit columns added by save_with_audit)
study_schema = StructType([
    StructField("study_id", StringType(), False),
    StructField("study_title", StringType(), True),
    StructField("study_nickname", StringType(), True),
    StructField("study_description", StringType(), True),
    StructField("study_phase", StringType(), True),
    StructField("status", StringType(), True),
    StructField("protocol_id", StringType(), True)
])

# Create DataFrame with study data
study_data = [(
    study_id,
    study_title,
    study_nickname or None,
    study_description or None,
    study_phase or None,
    "IN_PROGRESS",  # Initial status
    protocol_id if protocol_id else None
)]

df_study = spark.createDataFrame(study_data, study_schema)

# Use save_with_audit with merge mode for upsert behavior
save_with_audit(
    df=df_study,
    table_name=full_table_name,
    created_by_principal=created_by or "system",
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode="merge",
    merge_keys=["study_id"]
)
print(f"Study {study_id} saved successfully")

# Debug: Print table schema after save
print(f"\n=== Table Schema for {full_table_name} ===")
spark.sql(f"DESCRIBE {full_table_name}").show(truncate=False)


In [ ]:
# ============================================================================
# Create md_protocol_draft record in Silver schema
# ============================================================================
from clinical_data_standards_framework.utils import save_with_audit

SILVER_SCHEMA = "silver_md"
PROTOCOL_TABLE = "md_protocol_draft"
protocol_table_full = f"{catalog}.{SILVER_SCHEMA}.{PROTOCOL_TABLE}"

print(f"\n{'='*60}")
print(f"Creating Protocol Draft Record")
print(f"{'='*60}")
print(f"Target table: {protocol_table_full}")

# Create table if not exists - audit columns are added automatically by save_with_audit
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {protocol_table_full} (
        protocol_id STRING NOT NULL COMMENT 'Unique protocol identifier',
        study_id STRING NOT NULL COMMENT 'Associated study ID',
        document_id STRING COMMENT 'Document ID from md_file_history',
        protocol_title STRING COMMENT 'Protocol title (from study)',
        protocol_version STRING COMMENT 'Protocol version',
        status STRING COMMENT 'Status: PENDING, PARSING, SECTIONS_EXTRACTED, FAILED',
        section_count INT COMMENT 'Number of sections extracted',
        created_by_principal STRING COMMENT 'User who created the record',
        created_ts TIMESTAMP COMMENT 'Creation timestamp',
        last_updated_by_principal STRING COMMENT 'User who last updated',
        last_updated_ts TIMESTAMP COMMENT 'Last update timestamp',
        databricks_job_id STRING COMMENT 'Job ID for lineage',
        databricks_job_name STRING COMMENT 'Job name for lineage',
        databricks_run_id STRING COMMENT 'Run ID for lineage'
    ) USING DELTA
    COMMENT 'Protocol draft records linking studies to protocol documents'
""")
print(f"Table {protocol_table_full} ready")

# Only create protocol record if protocol_id is provided
if protocol_id:
    # Check if protocol already exists
    existing_protocol = spark.sql(f"SELECT protocol_id FROM {protocol_table_full} WHERE protocol_id = '{protocol_id}'")
    
    if existing_protocol.count() > 0:
        print(f"Protocol {protocol_id} already exists - updating status to PENDING...")
        spark.sql(f"""
            UPDATE {protocol_table_full}
            SET status = 'PENDING',
                last_updated_by_principal = '{created_by}',
                last_updated_ts = current_timestamp()
            WHERE protocol_id = '{protocol_id}'
        """)
    else:
        print(f"Creating new protocol draft: {protocol_id}")
        
        # Protocol schema - feature columns only (audit columns added by save_with_audit)
        protocol_schema = StructType([
            StructField("protocol_id", StringType(), False),
            StructField("study_id", StringType(), False),
            StructField("document_id", StringType(), True),
            StructField("protocol_title", StringType(), True),
            StructField("protocol_version", StringType(), True),
            StructField("status", StringType(), True),
            StructField("section_count", IntegerType(), True)
        ])
        
        protocol_data = [(
            protocol_id,
            study_id,
            None,  # document_id - will be set by protocol processor
            study_title,
            "1.0",  # Initial version
            "PENDING",  # Initial status
            0  # section_count - will be updated after parsing
        )]
        
        df_protocol = spark.createDataFrame(protocol_data, protocol_schema)
        
        # Use save_with_audit for consistent audit column management
        save_with_audit(
            df=df_protocol,
            table_name=protocol_table_full,
            created_by_principal=created_by or "system",
            databricks_job_id=databricks_job_id,
            databricks_job_name=databricks_job_name,
            databricks_run_id=databricks_run_id,
            mode="append"
        )
        print(f"Protocol draft {protocol_id} created successfully")
    
    # Display the created/updated protocol
    result_protocol = spark.sql(f"SELECT * FROM {protocol_table_full} WHERE protocol_id = '{protocol_id}'")
    display(result_protocol)
else:
    print("No protocol_id provided - skipping protocol draft creation")


In [ ]:
# Verify and display the created/updated study
result = spark.sql(f"SELECT * FROM {full_table_name} WHERE study_id = '{study_id}'")
display(result)

# Set output for downstream tasks
dbutils.notebook.exit(study_id)
